# Source Tissue Masks to Annotations

Status: complete

The source images are low resolution WSIs with binary masks for the tissue that were generated as part of the NFT detection project (a.k.a. YOLO Braak project). In this notebook we extract contours for those masks and push them as DSA annotations, to its WSI in the Emory-ADRC collection (megabrain server).

In [90]:
# Imports
import sys
sys.path.append('../..')

from pandas import read_csv
from os.path import join, splitext
import cv2 as cv
import numpy as np
import large_image
from tqdm import tqdm

from neurotk import login, imread
from neurotk.girder_utils import get_tile_metadata, get_annotations_documents
from neurotk.utils import contours_to_points

In [7]:
# Girder client active session.
# gc1 = login('https://computablebrain.emory.edu/api/v1', username='jvizcar')
gc = login('https://megabrain.neurology.emory.edu/api/v1', username='jvizcar')

Password for jvizcar:  ········


In [62]:
# Path to directories.
root_dir = '/jcDataStore/Data/nft-ai-project/wsi-inference'
mask_dir = join(root_dir, 'tissue-masks/masks')
sf = 40 / 0.25  # scale factor from low res mask to WSI scale
doc_name = 'htk-manual-tissue'

In [87]:
# Read the WSI information.
wsi_metadata = read_csv(join(root_dir, 'wsis.csv'))

# Loop for each WSI.
for _, r in tqdm(wsi_metadata.iterrows(), total=len(wsi_metadata)):
    # Look for this image in Megabrain.
    fp = f"/collection/Emory-ADRC/{r.filepath.split('/wsis/')[1]}"
    fp = fp.replace('/', '%2F')
    item = gc.get(f'resource/lookup?path={fp}')

    # Ignore if item of same name / filepath is not in Megabrain.
    if item:
        # Skip if the annotation document is already there.
        exist_flag = False
        
        for ann_doc in get_annotations_documents(gc, item['_id']):
            if ann_doc.get('annotation', {}).get('name') == doc_name:
                exist_flag = True
                break

        if exist_flag:
            continue
            
        # Get the filename without extension.
        fn = splitext(r.wsi_name)[0]
    
        # Read mask file.
        mask_fp = join(mask_dir, fn + '.png')
        mask = (imread(mask_fp)[:, :, 0] > 0).astype(np.uint8)
    
        # Extract contours.
        contours = cv.findContours(mask, cv.RETR_TREE, 
                                   cv.CHAIN_APPROX_SIMPLE)[0]

        smoothed_contours = []

        for contour in contours:
            smoothed_contours.append(cv.approxPolyDP(contour, 1, True))
    
        # Convert the list of contours to points in DSA format.
        tissue_points = contours_to_points(smoothed_contours)
    
        # Convert each contour into a list dictionary to pass as an annotation 
        # DSA element.
        tissue_els = []
    
        for pt in tissue_points:
            # Skip a point with too few points*
            # * DSA appears to prevent annotations of three points only.
            if len(pt) < 4:
                continue
                
            # Scale the points
            pt = np.array(pt) * sf
            
            tissue_els.append({
                'group': doc_name,
                'type': 'polyline',
                'lineColor': 'rgb(0,179,60)',
                'lineWidth': 4.0,
                'closed': True,
                'points': pt.tolist(),
                'label': {'value': doc_name},
            })

        # Push as annotations.
        _ = gc.post(
            f"/annotation?itemId={item['_id']}", 
            json={
                'name': doc_name, 
                'description': 'Extracted from low res binary masks.', 
                'elements': tissue_els})

100%|███████████████████████████████████████████████| 447/447 [03:58<00:00,  1.88it/s]
